### 제가 작성하는 모든 코드는 Github 저장소를 통해 공유드립니다.
※ 쉬는시간 또는 점심시간 또는 마치기 직전에~

#### 배깅(Bagging)
> * 분산 감소 -> 과적합 방지
> * 다수의 모델을 병렬처리 할 수도 있음

다수의 모델 -> 해석의 어려움, 연산량의 증가  
같은 데이터 포인트(sample, row)를 복수개의 모델에 학습(bootstrap sampling 때문)시키고 그 결과를 취합(aggregating)하게 되는 경우가 있음  
특정 모델에서 매우 튀는 값(이상치에 준하는)이 발생하더라도 다수의 모델의 결과를 취합하게 되기 때문에 그 편향(?)을 줄이는 효과가 있고, 여러 모델의 결과를 취합하면서 각 모델에서 발생하게되는 오차를 서로 상쇄하여 결과적으로 분산이 줄어드는 효과가 있음.

전반적으로 안정적인 결과(상대적으로 낮은 분산). 과적합 방지.

모델 개수만큼 에측 오차의 분산이 감소($n$: 모델 개수)
$$\frac{\sigma^2}{n}$$

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_s = pd.DataFrame(dict(col1 = [100, 200, 300], 
                         col2 = ["A", "B", "C"],
                         shape = ["a", "b", "c"]))
df_s

col1 col2 shape
0   100    A     a
1   200    B     b
2   300    C     c

In [3]:
df_s.col1

0    100
1    200
2    300
Name: col1, dtype: int64

In [4]:
df_s.shape

(3, 3)

In [5]:
df_s["shape"]

0    a
1    b
2    c
Name: shape, dtype: object

In [6]:
df_s.aaa = 123

In [8]:
df_s.aaa

123

In [9]:
df_s["aaa"] = 123

In [10]:
df_s

col1 col2 shape  aaa
0   100    A     a  123
1   200    B     b  123
2   300    C     c  123

In [11]:
df_s.iloc[:2, ]

col1 col2 shape  aaa
0   100    A     a  123
1   200    B     b  123

In [12]:
df_s.loc[:2, ]

col1 col2 shape  aaa
0   100    A     a  123
1   200    B     b  123
2   300    C     c  123

In [13]:
df_s.add_prefix("a_") # 기출 유형

a_col1 a_col2 a_shape  a_aaa
0     100      A       a    123
1     200      B       b    123
2     300      C       c    123

In [14]:
df_s.add_suffix("_x") # 기출 유형

col1_x col2_x shape_x  aaa_x
0     100      A       a    123
1     200      B       b    123
2     300      C       c    123

In [15]:
df_s.columns = "a_" + df_s.columns
df_s

a_col1 a_col2 a_shape  a_aaa
0     100      A       a    123
1     200      B       b    123
2     300      C       c    123

### 필터링
https://datadoctorblog.com/2023/04/08/Py-Basic-Pandas-Filtering/

In [17]:
# 시험장에서 이정도 난이도의 코드를 작성할 일이 아마... 없을겁니다.
df_s.loc[:, df_s.columns.str.contains("[0-9]$")] # 숫자로 끝나는 변수만 추출

a_col1 a_col2
0     100      A
1     200      B
2     300      C

In [ ]:
df_s.loc[df_s["a_col1"] == 100, ]

In [ ]:
df_s.loc[(df_s["a_col1"] == 100) | (df_s["a_col1"] == 300), ]

In [ ]:
condi_1 = df_s["a_col1"] == 100
condi_2 = df_s["a_col1"] == 300
df_s.loc[condi_1 | condi_2, ]

In [23]:
df_s.loc[df_s["a_col1"].isin([100, 300]), ]

a_col1 a_col2 a_shape  a_aaa
0     100      A       a    123
2     300      C       c    123

In [24]:
df = pd.read_csv("krx_202105_utf8.csv")
df.head(1)

지수명       종가     대비   등락률       시가       고가       저가        거래량  \
0  KRX 300  1901.05 -11.63 -0.61  1913.15  1929.94  1896.65  279548840   

             거래대금            상장시가총액        날짜  
0  13488481375036  2028800954161500  20210503

In [26]:
df_sub = df.loc[df["지수명"] == "KRX 300", ]
df_sub.head(2)

지수명       종가     대비   등락률       시가       고가       저가        거래량  \
0   KRX 300  1901.05 -11.63 -0.61  1913.15  1929.94  1896.65  279548840   
29  KRX 300  1916.40  15.35  0.81  1903.57  1916.40  1892.36  244040965   

              거래대금            상장시가총액        날짜  
0   13488481375036  2028800954161500  20210503  
29  11708937073166  2043945386926060  20210504

In [27]:
df_sub2 = df.loc[df["지수명"] == "KRX300", ]
df_sub2.head(2)

Empty DataFrame
Columns: [지수명, 종가, 대비, 등락률, 시가, 고가, 저가, 거래량, 거래대금, 상장시가총액, 날짜]
Index: []

In [28]:
df["지수명"].unique()

array(['KRX 300', 'KTOP 30', 'KRX 100', 'KRX Mid 200', 'KRX 자동차',
       'KRX 반도체', 'KRX 헬스케어', 'KRX 은행', 'KRX 에너지화학', 'KRX 철강', 'KRX 방송통신',
       'KRX 건설', 'KRX 증권', 'KRX 기계장비', 'KRX 보험', 'KRX 운송', 'KRX 경기소비재',
       'KRX 필수소비재', 'KRX 미디어&엔터테인먼트', 'KRX 정보기술', 'KRX 유틸리티',
       'KRX 300 정보기술', 'KRX 300 금융', 'KRX 300 자유소비재', 'KRX 300 산업재',
       'KRX 300 헬스케어', 'KRX 300 커뮤니케이션서비스', 'KRX 300 소재', 'KRX 300 필수소비재'],
      dtype=object)

In [33]:
# type(df.iloc[117, ])
df.iloc[117, ]

지수명                KTOP 30
종가                11869.29
대비                  135.96
등락률                   1.16
시가                11753.02
고가                11913.62
저가                11753.02
거래량               52051112
거래대금         5414226520140
상장시가총액    1241838527633600
날짜                20210510
Name: 117, dtype: object

In [34]:
df.iloc[[117], ]

지수명        종가      대비   등락률        시가        고가        저가       거래량  \
117  KTOP 30  11869.29  135.96  1.16  11753.02  11913.62  11753.02  52051112   

              거래대금            상장시가총액        날짜  
117  5414226520140  1241838527633600  20210510

In [35]:
# 최대값이 여러 행에 걸쳐서 있을 경우 해당 행이 모두 반환됨
df.loc[df["종가"] == df["종가"].max(), ]

지수명        종가      대비   등락률        시가        고가        저가       거래량  \
117  KTOP 30  11869.29  135.96  1.16  11753.02  11913.62  11753.02  52051112   

              거래대금            상장시가총액        날짜  
117  5414226520140  1241838527633600  20210510

In [ ]:
df["종가"].idxmax()

In [ ]:
df.loc[df["종가"].idxmax(), ]

In [ ]:
# 최대값이 여러 행에 걸쳐서 있을 경우 첫 번째 행이 반환됨
df.loc[[df["종가"].idxmax()], ]

In [36]:
import os
os.getcwd() # <-- 파일을 이 코드 실행결과로 나오는 경로에 위치

'C:\\Users\\Encaion\\Desktop\\python_class'

### 결측치 처리
https://datadoctorblog.com/2023/04/24/Py-Basic-Pandas-Missing/

In [39]:
df = pd.read_csv("iris_missing.csv")
df.head(2)

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
0           5.1          NaN           NaN          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa

In [49]:
df_sub = df.head()
df_sub.info() # 어쩌다가... 안될수도 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  5 non-null      float64
 1   Sepal.Width   4 non-null      float64
 2   Petal.Length  4 non-null      float64
 3   Petal.Width   5 non-null      float64
 4   Species       5 non-null      object 
dtypes: float64(4), object(1)
memory usage: 328.0+ bytes


In [43]:
df_sub.count()

Sepal.Length    5
Sepal.Width     4
Petal.Length    4
Petal.Width     5
Species         5
dtype: int64

In [44]:
df_sub.dtypes

Sepal.Length    float64
Sepal.Width     float64
Petal.Length    float64
Petal.Width     float64
Species          object
dtype: object

In [45]:
len(df_sub) - df_sub.count() # 각 변수별 결측치 개수 계산....이지만 거의 안씀

Sepal.Length    0
Sepal.Width     1
Petal.Length    1
Petal.Width     0
Species         0
dtype: int64

In [41]:
df_sub.isna().sum() # 🌟🌟🌟🌟

Sepal.Length    0
Sepal.Width     1
Petal.Length    1
Petal.Width     0
Species         0
dtype: int64

In [50]:
df_sub.loc[df_sub["Sepal.Width"].isna(), ]

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
0           5.1          NaN           NaN          0.2  setosa

In [51]:
df_sub.loc[df_sub["Sepal.Width"].notna(), ]

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa

다음 코드 실행시 발생하는 빨간색 메세지는 에러가 아닌 경고.  
해당 경고는 .copy() 또는 .reset_index() 등으로 해결할 수 있음.

In [54]:
df_sub["Sepal.Width"] = df_sub["Sepal.Width"].fillna(-999)
df_sub["Petal.Length"] = df_sub["Petal.Length"].fillna(df_sub["Petal.Length"].mean())
# 산술연산 수행시 기본적으로 Pandas는 결측치를 제외하고 계산

C:\Users\Encaion\AppData\Local\Temp\ipykernel_29940\3187634281.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["Sepal.Width"] = df_sub["Sepal.Width"].fillna(-999)
C:\Users\Encaion\AppData\Local\Temp\ipykernel_29940\3187634281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sub["Petal.Length"] = df_sub["Petal.Length"].fillna(df_sub["Petal.Length"].mean())


In [53]:
df_sub.dropna()

Sepal.Length  Sepal.Width  Petal.Length  Petal.Width Species
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa

In [ ]:
# df_sub2 = df.head().copy()
df_sub2 = df.head().reset_index(drop = True)
df_sub2["Sepal.Width"] = df_sub2["Sepal.Width"].fillna(-999)
df_sub2["Petal.Length"] = df_sub2["Petal.Length"].fillna(df_sub2["Petal.Length"].mean())
df_sub2

In [ ]:
df_sub2.reset_index() # .groupby() 메서드와 많이 사용

특정 변수를 기존 위치에 관계없이 가장 앞으로 옮길 수 있음  
머신러닝 모델링 시 종속변수를 가장 앞으로 옮기기 좋음.

In [ ]:
df_sub.set_index("Species").reset_index()

### 정렬
https://datadoctorblog.com/2023/04/25/Py-Basic-Pandas-Sorting/

시험장에서 임의로 정렬한 데이터를 사용하여 분석하지 말것. ❌❌❌  
대부분의 머신러닝 알고리즘에는 랜덤이 들어가있기 때문에 변수 순서 또는 행 순서가 변경되는 경우 다른 답이 도출됨.

### 범주형 데이터의 탐색
https://datadoctorblog.com/2023/04/04/Py-Basic-Pandas-Counting/

In [64]:
df = pd.read_csv("bike.csv")
df.head(2)

datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40

In [69]:
df["season"].unique()

array([1, 2, 3, 4], dtype=int64)

In [70]:
df["season"].nunique()

4

In [71]:
df["season"].value_counts()

season
4    2734
2    2733
3    2733
1    2686
Name: count, dtype: int64

In [72]:
df["season"].value_counts(normalize = True)

season
4    0.251148
2    0.251056
3    0.251056
1    0.246739
Name: proportion, dtype: float64

In [73]:
pd.crosstab(df["season"], df["holiday"])

holiday     0   1
season           
1        2615  71
2        2685  48
3        2637  96
4        2638  96

https://datadoctorblog.com/2023/05/06/Py-Basic-Pandas-Crosstab/#%EC%A0%95%EA%B7%9C%ED%99%94

In [74]:
pd.crosstab(df["season"], df["holiday"], normalize = "index")

holiday         0         1
season                     
1        0.973567  0.026433
2        0.982437  0.017563
3        0.964874  0.035126
4        0.964887  0.035113

In [81]:
pd.crosstab(df["season"], df["holiday"], 
            values = df["count"], aggfunc = "max")

holiday    0    1
season           
1        801  228
2        873  712
3        977  584
4        948  596

In [82]:
df.groupby(["season", "holiday"])["count"].max()

season  holiday
1       0          801
        1          228
2       0          873
        1          712
3       0          977
        1          584
4       0          948
        1          596
Name: count, dtype: int64

In [84]:
df.pivot_table(index = "season", columns = "holiday", 
               values = "count", aggfunc = "max")

holiday    0    1
season           
1        801  228
2        873  712
3        977  584
4        948  596

### .groupby()
https://datadoctorblog.com/2023/04/28/Py-Basic-Pandas-Groupby/

In [ ]:
df.groupby(["season", "holiday"])[["count", "temp"]].max()

In [ ]:
df.groupby("season")[["count", "temp"]].max().reset_index()

In [ ]:
df.groupby("season")[["count", "temp"]].agg({"count": "max", 
                                             "temp": "min"}).reset_index() # 기출유형 🙆‍♂️

In [ ]:
df_g1 = df.groupby("season")["count"].max().reset_index()
df_g2 = df.groupby("season")["temp"].min().reset_index()
# df_g_join = df_g1.merge(df_g2, on = "season")
df_g_join = pd.merge(df_g1, df_g2, on = "season")
df_g_join

In [94]:
df_g1.set_index("season").join(df_g2.set_index("season")).reset_index()

season  count   temp
0       1    801   0.82
1       2    873   9.84
2       3    977  15.58
3       4    948   5.74

> **기출 유형**: 날짜별 등락률의 값이 최대값에 해당하는 지수명

https://datadoctorblog.com/2023/05/22/Py-Basic-Pandas-Merge

In [96]:
df = pd.read_csv("krx_202105_utf8.csv")
df.head(2)

지수명        종가     대비   등락률        시가        고가        저가        거래량  \
0  KRX 300   1901.05 -11.63 -0.61   1913.15   1929.94   1896.65  279548840   
1  KTOP 30  11520.28 -86.30 -0.74  11609.44  11702.14  11490.84   50048566   

             거래대금            상장시가총액        날짜  
0  13488481375036  2028800954161500  20210503  
1   5645660149606  1208343758559140  20210503

In [ ]:
df_g_max = df.groupby("날짜")["등락률"].max().reset_index()
df_g_max.head(2)

In [ ]:
df_g_join = pd.merge(df_g_max, df, on = ["날짜", "등락률"], how = "inner")
df_g_join.head(2)

In [102]:
df_g_join = pd.merge(df_g_max, df[["날짜", "등락률", "지수명"]], on = ["날짜", "등락률"], how = "inner")
df_g_join.head(2)

날짜   등락률        지수명
0  20210503  2.12    KRX 자동차
1  20210504  1.92  KRX 에너지화학

### 문자 데이터 핸들링
https://datadoctorblog.com/2023/04/20/Py-Basic-Pandas-String-Data/

In [103]:
df_s = pd.DataFrame(dict(col1 = [100, 200, 300], 
                         col2 = ["abc", "bbb", "ccc"],
                         col3 = [123, "1234-5678", "010-1234-5678"]))
df_s

col1 col2           col3
0   100  abc            123
1   200  bbb      1234-5678
2   300  ccc  010-1234-5678

In [ ]:
df_s["col2"].replace("a", "🎄")

In [110]:
df_s["col2"].replace("abc", "🎄")

0      🎄
1    bbb
2    ccc
Name: col2, dtype: object

In [111]:
df_s["col2"].replace({"abc": "🎄", "ccc": "✨"})

0      🎄
1    bbb
2      ✨
Name: col2, dtype: object

In [112]:
df_s["col2"].str.replace("a", "🎄")

0    🎄bc
1    bbb
2    ccc
Name: col2, dtype: object

In [113]:
df_s["col2"].str.replace("b", "🎄")

0    a🎄c
1    🎄🎄🎄
2    ccc
Name: col2, dtype: object

In [114]:
df_s["col2"].str.replace("bbb", "🎄")

0    abc
1      🎄
2    ccc
Name: col2, dtype: object

In [115]:
df_s["col2"].str.contains("a")

0     True
1    False
2    False
Name: col2, dtype: bool

In [118]:
df_s.loc[df_s["col2"].str.contains("a"), ]

col1 col2 col3
0   100  abc  123

In [119]:
~df_s["col2"].str.contains("a") # True/False 반전

0    False
1     True
2     True
Name: col2, dtype: bool

In [120]:
df_s.loc[~df_s["col2"].str.contains("a"), ] # "a"가 없는.

col1 col2           col3
1   200  bbb      1234-5678
2   300  ccc  010-1234-5678

In [ ]:
df_s.loc[df_s["col2"].str.contains("^[^a]*?$"), ] # 시험에서 이렇게 어려운 정규식은 안씀.

In [125]:
df_s.dtypes

col1     int64
col2    object
col3    object
dtype: object

In [124]:
df_s["col3"].str.split("-")

0                  NaN
1         [1234, 5678]
2    [010, 1234, 5678]
Name: col3, dtype: object

In [126]:
df_s["col3"].astype("str").str.split("-")

0                [123]
1         [1234, 5678]
2    [010, 1234, 5678]
Name: col3, dtype: object

In [127]:
df_s["col3"].astype("str").str.split("-").explode() # 리스트 구조를 폭🌟파🌟 (해체)

0     123
1    1234
1    5678
2     010
2    1234
2    5678
Name: col3, dtype: object

In [128]:
df_s["col3"].astype("str").str.split("-", expand = True)

0     1     2
0   123  None  None
1  1234  5678  None
2   010  1234  5678

In [129]:
df_s_split = df_s["col3"].astype("str").str.split("-", expand = True)
df_s_split.fillna("😮")

0     1     2
0   123     😮     😮
1  1234  5678     😮
2   010  1234  5678

In [130]:
df_s_split.add_prefix("col3_")

col3_0 col3_1 col3_2
0    123   None   None
1   1234   5678   None
2    010   1234   5678

In [131]:
df_s_split.columns = "col3_" + df_s_split.columns.astype("str")
df_s_split

col3_0 col3_1 col3_2
0    123   None   None
1   1234   5678   None
2    010   1234   5678

In [132]:
df = pd.read_csv("bike.csv")
df.head(2)

datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40

In [133]:
df1 = df.drop(columns = ["temp", "atemp"])
df2 = df[["temp", "atemp"]]
df2 = df2.add_suffix("_x")
df_cat = pd.concat([df1, df2], axis = 1)
df_cat.head(2)

datetime  season  holiday  workingday  weather  humidity  \
0  2011-01-01 00:00:00       1        0           0        1        81   
1  2011-01-01 01:00:00       1        0           0        1        80   

   windspeed  casual  registered  count  temp_x  atemp_x  
0        0.0       3          13     16    9.84   14.395  
1        0.0       8          32     40    9.02   13.635

In [137]:
df_s["col3"].astype("str").str.len()

0     3
1     9
2    13
Name: col3, dtype: int64

#### Q. 주어진 데이터는 두 물류창고에 적재된 물건과 관련된 데이터이다. 그런데 분석에 필요한 것은 A물류창고의 데이터이며 해당 물류창고의 데이터는 적재번호(lot_no)가 숫자와 문자 조합으로 총 10자리 고유번호로 지정되어 있는 반면 B물류창고는 같은 규칙으로 총 12자리 고유번호로 지정되어있다. 이를 참고로 하여 A물류창고의 데이터만 추출한 후 다음 분석을 실시하시오.

시험에서 .str.extract() 를 사용해야 하는 전처리 문제가 출제된 적이 있음  
https://datadoctorblog.com/2023/08/01/Py-ML-Split-Data-Sets-02/#%EA%B3%A0%EA%B8%89-%EC%88%AB%EC%9E%90-%EC%B6%94%EC%B6%9C%ED%98%95

### 시간 데이터 핸들링
https://datadoctorblog.com/2023/04/14/Py-Basic-Pandas-Time-Data/

In [139]:
ser_t1 = pd.Series(["2023-12-06", "2023-12-07", "2023-12-08"])
ser_t2 = pd.Series(["2023년 12월 6일", "2023년 12월 7일", "2023년 12월 8일"])
ser_t3 = pd.Series([12345, 23456, 34567])

In [ ]:
pd.to_datetime(ser_t1)

In [ ]:
# pd.to_datetime(ser_t2) # 에러!

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

In [145]:
pd.to_datetime(ser_t2, format = "%Y년 %m월 %d일")

0   2023-12-06
1   2023-12-07
2   2023-12-08
dtype: datetime64[ns]

In [146]:
pd.to_datetime(ser_t3)

0   1970-01-01 00:00:00.000012345
1   1970-01-01 00:00:00.000023456
2   1970-01-01 00:00:00.000034567
dtype: datetime64[ns]

In [147]:
pd.to_datetime(ser_t3, origin = "2023-12-06")

0   2023-12-06 00:00:00.000012345
1   2023-12-06 00:00:00.000023456
2   2023-12-06 00:00:00.000034567
dtype: datetime64[ns]

In [148]:
pd.to_datetime(ser_t3, origin = "2023-12-06", unit = "s")

0   2023-12-06 03:25:45
1   2023-12-06 06:30:56
2   2023-12-06 09:36:07
dtype: datetime64[ns]

In [151]:
df = pd.read_csv("bike.csv")
df_dt = df[["datetime"]].copy()
df_dt["datetime"] = pd.to_datetime(df_dt["datetime"])
df_dt["year"] = df_dt["datetime"].dt.year
df_dt["month"] = df_dt["datetime"].dt.month
df_dt["yyyymm"] = df_dt["datetime"].dt.strftime("%Y-%m") # 기출유형
df_dt["date"] = df_dt["datetime"].dt.date
df_dt["wday"] = df_dt["datetime"].dt.weekday # 5, 6이 토, 일
df_dt["wend"] = np.where(df_dt["wday"] >= 5, 1, 0) # 주말이 1
df_dt["hour"] = df_dt["datetime"].dt.hour
df_dt["shift_9h"] = df_dt["datetime"] + pd.to_timedelta(9, unit = "h")
df_dt.head(1)

datetime  year  month   yyyymm        date  wday  wend  hour  \
0 2011-01-01  2011      1  2011-01  2011-01-01     5     1     0   

             shift_9h  
0 2011-01-01 09:00:00

In [ ]:
df_dt2 = df[["datetime", "count"]].copy()
df_dt2["datetime"] = pd.to_datetime(df_dt2["datetime"])
df_dt2["date"] = df_dt2["datetime"].dt.date
df_dt2.head(2)

In [ ]:
df_g = df_dt2.groupby("date")["count"].sum().reset_index()
df_g2 = df_g.head(8).copy()
df_g2.iloc[4, 1] = np.nan
df_g2

#### Q. \[기출\] 변수 "count"에있는 결측치를 결측치 직전 값 1개와 직후 값 1개의 평균으로 대치하시오.
※ 단, 결측치는 2개 이상 연속으로 존재하지 않음.  

In [164]:
df_g2["s1"] = df_g2["count"].shift(1)
df_g2["s2"] = df_g2["count"].shift(2)
df_g2["sn1"] = df_g2["count"].shift(-1)
df_g2["fill1"] = np.where(df_g2["count"].isna(), (df_g2["s1"] + df_g2["sn1"]) / 2, df_g2["count"])
df_g2["fill2"] = df_g2["count"].interpolate()
df_g2

date   count      s1      s2     sn1   fill1   fill2
0  2011-01-01   985.0     NaN     NaN   801.0   985.0   985.0
1  2011-01-02   801.0   985.0     NaN  1349.0   801.0   801.0
2  2011-01-03  1349.0   801.0   985.0  1562.0  1349.0  1349.0
3  2011-01-04  1562.0  1349.0   801.0     NaN  1562.0  1562.0
4  2011-01-05     NaN  1562.0  1349.0  1606.0  1584.0  1584.0
5  2011-01-06  1606.0     NaN  1562.0  1510.0  1606.0  1606.0
6  2011-01-07  1510.0  1606.0     NaN   959.0  1510.0  1510.0
7  2011-01-08   959.0  1510.0  1606.0     NaN   959.0   959.0

### concat()
https://datadoctorblog.com/2023/05/22/Py-Basic-Pandas-Merge/#concat

In [165]:
df.head(2)

datetime  season  holiday  workingday  weather  temp   atemp  \
0  2011-01-01 00:00:00       1        0           0        1  9.84  14.395   
1  2011-01-01 01:00:00       1        0           0        1  9.02  13.635   

   humidity  windspeed  casual  registered  count  
0        81        0.0       3          13     16  
1        80        0.0       8          32     40

In [166]:
df1 = df.iloc[:3, 5:8]
df2 = df.iloc[-3:, 6:9]

In [170]:
pd.concat([df1, df2])

temp   atemp  humidity  windspeed
0      9.84  14.395        81        NaN
1      9.02  13.635        80        NaN
2      9.02  13.635        80        NaN
10883   NaN  15.910        61    15.0013
10884   NaN  17.425        61     6.0032
10885   NaN  16.665        66     8.9981

In [171]:
pd.concat([df1, df2], axis = 1)

temp   atemp  humidity   atemp  humidity  windspeed
0      9.84  14.395      81.0     NaN       NaN        NaN
1      9.02  13.635      80.0     NaN       NaN        NaN
2      9.02  13.635      80.0     NaN       NaN        NaN
10883   NaN     NaN       NaN  15.910      61.0    15.0013
10884   NaN     NaN       NaN  17.425      61.0     6.0032
10885   NaN     NaN       NaN  16.665      66.0     8.9981

In [172]:
pd.concat([df1, df2.reset_index(drop = True)], axis = 1)

temp   atemp  humidity   atemp  humidity  windspeed
0  9.84  14.395        81  15.910        61    15.0013
1  9.02  13.635        80  17.425        61     6.0032
2  9.02  13.635        80  16.665        66     8.9981

In [173]:
pd.concat([df1, df2.reset_index(drop = True).add_suffix("_y")], axis = 1)

temp   atemp  humidity  atemp_y  humidity_y  windspeed_y
0  9.84  14.395        81   15.910          61      15.0013
1  9.02  13.635        80   17.425          61       6.0032
2  9.02  13.635        80   16.665          66       8.9981

lambda 함수의 사용처
* crosstab()
* .pivot_table()
* .agg()
* .apply()

lambda 함수는 사용자정의 함수로 대체 가능.

In [ ]:
df_i = pd.read_csv("iris.csv")
df_i.head(2)

In [178]:
df_i["new1"] = df_i.iloc[:, :3].apply(lambda x: x.sum() + 100, axis = 1)

def sum_100(x):
    return x.sum() + 100

df_i["new2"] = df_i.iloc[:, :3].apply(sum_100, axis = 1)

In [179]:
df_i.head(2)

SepalLength  SepalWidth  PetalLength  PetalWidth Species   new1   new2
0          5.1         3.5          1.4         0.2  setosa  110.0  110.0
1          4.9         3.0          1.4         0.2  setosa  109.3  109.3

In [182]:
df_i.iloc[:3, :3]

SepalLength  SepalWidth  PetalLength
0          5.1         3.5          1.4
1          4.9         3.0          1.4
2          4.7         3.2          1.3

In [184]:
df_i.iloc[:3, :3].sum()

SepalLength    14.7
SepalWidth      9.7
PetalLength     4.1
dtype: float64

In [185]:
df_i.iloc[:3, :3].sum(axis = 1)

0    10.0
1     9.3
2     9.2
dtype: float64

### pivoting
https://datadoctorblog.com/2023/05/10/Py-Basic-Pandas-Pivoting/

In [ ]:
df_e = pd.read_csv("elec_load_2017_7d.csv")
df_e.head(2)

In [232]:
df_e_melt = df_e.melt(id_vars = ["YEAR", "MONTH", "DAY"])
df_e_melt.head(2)

YEAR  MONTH  DAY variable  value
0  2017      1    1     X1HR  60178
1  2017      1    2     X1HR  57067

In [233]:
df_e_melt = df_e.melt(id_vars = ["YEAR", "MONTH", "DAY"],
                      var_name = "HOUR", value_name = "LOAD")
df_e_melt.head(2)

YEAR  MONTH  DAY  HOUR   LOAD
0  2017      1    1  X1HR  60178
1  2017      1    2  X1HR  57067

### 머신러닝

* .predict(): 확률값이 가장 높은 범주를 반환
* .predict_proba(): 각 범주에 대한 확률값을 반환(2차원 Array 객체)

In [ ]:
import os
[f for f in os.listdir() if f[-3:] == "csv"]

In [ ]:
df_lr = pd.read_csv("logistic_regression_01.csv")
df_lr.head(2)

In [194]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [195]:
df_train, df_test = train_test_split(df_lr, train_size = 0.7, random_state = 123)
len(df_train), len(df_test)

(280, 120)

In [ ]:
model_lr = LogisticRegression(random_state = 123)
model_lr.fit(X = df_train.drop(columns = "admit"),
             y = df_train["admit"])
pred = model_lr.predict_proba(df_test.drop(columns = "admit"))

In [202]:
# pred # 분류하려는 범주의 개수만큼 열이 생성됨
pred[:5, 1]

array([0.57119336, 0.31389334, 0.4676319 , 0.14683109, 0.20751251])

In [228]:
pred_class1 = np.where(pred[:, 1] > 0.5, 1, 0)
pred_class2 = np.where(pred[:, 1] > 0.65, 1, 0)

In [235]:
pred_class1

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [209]:
from sklearn.metrics import roc_auc_score

# aa = roc_auc_score(y_true = df_test["admit"], y_score = pred[:, 1])
aa

0.7621179376350725

In [215]:
# bb = roc_auc_score(y_true = df_test["admit"], y_score = pred_class1)
bb

0.5790367397344859

In [216]:
from sklearn.metrics import precision_score

In [219]:
# cc = precision_score(y_true = df_test["admit"], y_pred = pred_class1)
cc

0.875

In [230]:
# dd = precision_score(y_true = df_test["admit"], y_pred = pred_class2)
dd

0.0